In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_gen = sps.hstack([ICM1, ICM2])
ICM_all = sps.hstack([ICM1, ICM2, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

seed = 2509
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.7, seed=seed)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [4]:
import os

ofp = "../models_temp/Similarity_Hybrid/seed" + str(seed) + "/"

models_to_combine_best = {
                            'EASE': {"topK": None, "normalize_matrix": False, "l2_norm": 3749.2876352484},
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' :  {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_all' : {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'SLIM_all' : {'l1_ratio': 0.0001043005302985496, 'topK': 1135, 'alpha': 0.06322495726710943, 'workers': 8, 'mw': 1.2937257828415842},
                            'SLIM_all2' : {'topK': 4997, 'l1_ratio': 0.00036065147205858187, 'alpha': 0.03203200564199538, 'mw': 1.0785132020354402, 'workers': 8},
                            'UserKNN' : {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'SLIM_BPR_Hyb' : {"epochs": 1443, "lambda_i": 8.900837513818856e-05, "lambda_j": 1.2615223007492727e-05, "learning_rate": 0.0037706733838839264, "topK": 6181, "random_seed": 1234, "sgd_mode": "sgd"},
                            'Neural': {"epochs": 295, "learning_rate": 9.598708662230564e-05, "l2_reg": 1.4122381520119457e-05, "dropout": 0.5759445339123964, "total_anneal_steps": 147387, "anneal_cap": 0.4376000433422989, "batch_size": 256, "encoding_size": 185, "next_layer_size_multiplier": 6, "max_n_hidden_layers": 1, "max_layer_size": 5000.0},
                            'EASE_stk': {"topK": None, "normalize_matrix": False, 'mw': 4, "l2_norm": 3749.2876352484}
                         }


In [5]:
def model_init(recommender, name, args, seed):
    path_name = ofp + name + str(seed) + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name + str(seed))
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name + str(seed))

In [6]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender_Hybrid
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython_Hybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender

In [7]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.HybridScores.DifferentStructure import TwoDifferentModelRecommender
from Recommenders.HybridScores.DifferentStructure import ThreeDifferentModelRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender_FAST

In [8]:
groups = ld.load_group()

In [9]:
recommender_slim_all = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
recommender_slim_all2 = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
#recommender_slim_wei = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_weighted)
recommender_ease = EASE_R_Recommender(URM_train)
#recommender_itemknn = ItemKNNCBFRecommender(URM_train, ICM_all)
recommender_rp3 = RP3betaRecommender(URM_train)
#recommender_svd = PureSVDRecommender(URM_train)
recommender_user = UserKNNCFRecommender(URM_train)

model_init(recommender_slim_all, 'SLIMall', models_to_combine_best['SLIM_all'], seed)
model_init(recommender_slim_all2, 'SLIMall2', models_to_combine_best['SLIM_all2'], seed)
#model_init(recommender_slim_wei, 'SLIMweig', models_to_combine_best['SLIMweig'], seed)
model_init(recommender_ease, 'EASE', models_to_combine_best['EASE'], seed)
model_init(recommender_rp3, 'RP3beta', models_to_combine_best['RP3beta'], seed)
#model_init(recommender_itemknn, 'ICM_all', models_to_combine_best['icm_all'], seed)
#model_init(recommender_svd, 'PureSVD', {'num_factors': 30}, seed)
model_init(recommender_user, 'UserKNN', models_to_combine_best['UserKNN'], seed)

recommender_slim_all.RECOMMENDER_NAME = 'SLIMall'
recommender_slim_all2.RECOMMENDER_NAME = 'SLIMall2'
#recommender_slim_wei.RECOMMENDER_NAME = 'SLIMweig'
recommender_ease.RECOMMENDER_NAME = 'EASE'
#recommender_itemknn.RECOMMENDER_NAME = 'KNN'
recommender_rp3.RECOMMENDER_NAME = 'RP3'
#recommender_svd.RECOMMENDER_NAME = 'SVD'

SLIMElasticNetRecommender: URM Detected 1 ( 0.0%) items with no interactions.
SLIMElasticNetRecommender: URM Detected 1 ( 0.0%) items with no interactions.
EASE_R_Recommender: URM Detected 1 ( 0.0%) items with no interactions.
RP3betaRecommender: URM Detected 1 ( 0.0%) items with no interactions.
UserKNNCFRecommender: URM Detected 1 ( 0.0%) items with no interactions.
../models_temp/Similarity_Hybrid/seed2509/SLIMall2509.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed2509/SLIMall2509'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed2509/SLIMall22509.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed2509/SLIMall22509'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed2509/EASE2509.zip
Model found!
EASE_R_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/seed2509/EASE2509'
EASE_R_Recommender

In [10]:
#recommender_ials = IALSRecommender(URM_train)
recommender_bpr = SLIM_BPR_Cython(URM_train)
#recommender_bpr_hyb = SLIM_BPR_Cython_Hybrid(URM_train, ICM_weighted)
recommender_ials_hyb = IALSRecommender_Hybrid(URM_train, ICM_weighted)
#recommender_p3alpha = P3alphaRecommender(URM_train)
recommender_slimer = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender_ease_hyb = EASE_R_Recommender_Hybrid(URM_train, ICM_all)

model_init(recommender_bpr, 'SLIMBPR', models_to_combine_best['SLIMBPR'], seed)
#model_init(recommender_slim_bpr_hyb, 'SLIMBPR_Hyb6180', models_to_combine_best['SLIM_BPR_Hyb'])
model_init(recommender_ials_hyb, 'IALS_Hyb', models_to_combine_best['IALS_Hyb'], seed)
#model_init(recommender_p3alpha, 'P3alpha6180', models_to_combine_best['P3alpha'])
model_init(recommender_slimer, 'SLIMER', models_to_combine_best['SLIMER'], seed)
#model_init(recommender_ials, 'IALS6180', models_to_combine_best['IALS'])
model_init(recommender_ease_hyb, 'EASE_stacked', models_to_combine_best['EASE_stk'], seed)

recommender_bpr.RECOMMENDER_NAME = 'SLIMall'
#recommender_slim_bpr_hyb.RECOMMENDER_NAME = 'SLIMall2'
recommender_ials_hyb.RECOMMENDER_NAME = 'IALS_Hyb'
#recommender_p3alpha.RECOMMENDER_NAME = 'P3alpha'
recommender_slimer.RECOMMENDER_NAME = 'SLIMER'
#recommender_ials.RECOMMENDER_NAME = 'IALS'
recommender_ease_hyb.RECOMMENDER_NAME = 'EASE_Hybrid'

SLIM_BPR_Recommender: URM Detected 1 ( 0.0%) items with no interactions.
SLIMElasticNetRecommender: URM Detected 1 ( 0.0%) items with no interactions.
EASE_R_Recommender_Hybrid: URM Detected 1 ( 0.0%) items with no interactions.
../models_temp/Similarity_Hybrid/seed2509/SLIMBPR2509.zip
Model found!
SLIM_BPR_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/seed2509/SLIMBPR2509'
SLIM_BPR_Recommender: Loading complete
../models_temp/Similarity_Hybrid/seed2509/IALS_Hyb2509.zip
Model found!
IALSRecommender_Hybrid: Loading model from file '../models_temp/Similarity_Hybrid/seed2509/IALS_Hyb2509'
IALSRecommender_Hybrid: Loading complete
../models_temp/Similarity_Hybrid/seed2509/SLIMER2509.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed2509/SLIMER2509'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed2509/EASE_stacked2509.zip
Model found!
EASE_R_Recommender_Hybrid: Loading model from

In [11]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])
        
def rank_models_group_wise(evaluators, recommenders):
    i = 0 # group
    dict_res = {}
    for e in evaluators:
        dict_res[i] = {}
        for r in recommenders:
            r_d, _ = e.evaluateRecommender(r)
            dict_res[i][r.RECOMMENDER_NAME] = r_d.loc[10]['MAP']
        i+=1
    return dict_res

In [12]:
def rank_models_all(evaluator, recommenders):
    dict_res = {}
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        dict_res[r.RECOMMENDER_NAME] = r_d.loc[10]['MAP']
        
    return dict_res

In [13]:
import numpy as np

evaluators = []
n_groups = 3
cutoff = 10
n_users = URM_all.shape[0]

for group_id in range(0, n_groups):
    users_in_group = groups[group_id]
    print("GROUP:", group_id, "NUMBER:", len(users_in_group))
    all_users = np.arange(0, n_users)
    users_not_in_group = np.delete(all_users, users_in_group)

    evaluator_test = EvaluatorHoldout(URM_validation, cutoff_list=[cutoff], ignore_users=users_not_in_group)
    evaluators.append(evaluator_test)

GROUP: 0 NUMBER: 5442
EvaluatorHoldout: Ignoring 8208 Users
GROUP: 1 NUMBER: 6560
EvaluatorHoldout: Ignoring 7090 Users
GROUP: 2 NUMBER: 1648
EvaluatorHoldout: Ignoring 12002 Users


In [14]:
recommender_slim_all2_ease = TwoDifferentModelRecommender(URM_train, recommender_slim_all2, recommender_ease)
recommender_slim_all2_ease.fit(None, 0.6)
recommender_slim_all2_ease.RECOMMENDER_NAME = "SLIM_ALL2+EASE"

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.


In [15]:
recommender_slim_all_ease = TwoDifferentModelRecommender(URM_train, recommender_slim_all, recommender_ease)
recommender_slim_all_ease.fit(None, 0.6)
recommender_slim_all_ease.RECOMMENDER_NAME = "SLIM_ALL+EASE"

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.


In [16]:
recommender_slimer_ease = TwoDifferentModelRecommender(URM_train, recommender_slimer, recommender_ease)
recommender_slimer_ease.fit(None, 0.605)
recommender_slimer_ease.RECOMMENDER_NAME = "SLIMER+EASE"

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.


In [17]:
recommender_slim_all2_ease_stk = TwoDifferentModelRecommender(URM_train, recommender_slim_all2, recommender_ease_hyb)
recommender_slim_all2_ease_stk.fit(None, 0.532)
recommender_slim_all2_ease_stk.RECOMMENDER_NAME = "SLIM_ALL2+EASE_HYB"

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.


In [18]:
recommender_slim_all_ease_stk = TwoDifferentModelRecommender(URM_train, recommender_slim_all, recommender_ease_hyb)
recommender_slim_all_ease_stk.fit(None, 0.532)
recommender_slim_all_ease_stk.RECOMMENDER_NAME = "SLIM_ALL+EASE_HYB"

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.


In [19]:
recommender_slimer_ease_stk = TwoDifferentModelRecommender(URM_train, recommender_slimer, recommender_ease_hyb)
recommender_slimer_ease_stk.fit(None, 0.588)
recommender_slimer_ease_stk.RECOMMENDER_NAME = "SLIMER+EASE_HYB"

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.


In [20]:
best_hyp = {'norm' : 2, 'alpha': 0.9822105840347196, 'beta': 0.3027796526609026, 'gamma': 0.11380418938181623}
recommender_ials_slimer_ease = ThreeDifferentModelRecommender(URM_train, recommender_slimer, recommender_ease, recommender_ials_hyb)
recommender_ials_slimer_ease.fit(**best_hyp)
recommender_ials_slimer_ease.RECOMMENDER_NAME = 'SLIMER+ease+IALS'

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.7021836559565641
EASE with weight beta: 0.2164575773367719
IALS_Hyb with weight gamma: 0.08135876670666396
Norm type: 2


In [21]:
best_hyp_2 = {'norm' : None, 'alpha': 0.9834878049427671, 'beta': 0.44912206809137145, 'gamma': 0.3385233480006371}
recommender_rp3_slimer_ease = ThreeDifferentModelRecommender(URM_train, recommender_slimer, recommender_ease, recommender_rp3)
recommender_rp3_slimer_ease.fit(**best_hyp_2)
recommender_rp3_slimer_ease.RECOMMENDER_NAME = 'SLIMER+ease+RP3'

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.5552873116840806
EASE with weight beta: 0.2535789305724694
RP3 with weight gamma: 0.19113375774345
Norm type: None


In [22]:
best_hyp_3 = {'norm' : None, 'alpha': 0.862073716293543, 'beta': 0.356861076175057, 'gamma': 0.4541302980059095}
recommender_rp3_slimer_ease_stk = ThreeDifferentModelRecommender(URM_train, recommender_slimer, recommender_ease_hyb, recommender_rp3)
recommender_rp3_slimer_ease_stk.fit(**best_hyp_3)
recommender_rp3_slimer_ease_stk.RECOMMENDER_NAME = 'SLIMER+ease_stk+RP3'

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.5152660952653338
EASE_Hybrid with weight beta: 0.2132977839337053
RP3 with weight gamma: 0.27143612080096086
Norm type: None


In [23]:
recommender_slimer_bpr_ease = ThreeDifferentModelRecommender(URM_train.tocsr(), recommender_slimer, recommender_bpr, recommender_ease)
recommender_slimer_bpr_ease.fit(2, 0.9805415410671916, 0.9992499197990959, 0.3670862415112377)
recommender_slimer_bpr_ease.RECOMMENDER_NAME = 'SLIMER+BPR+ease'

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.4178068333402484
SLIMall with weight beta: 0.4257784369363589
EASE with weight gamma: 0.15641472972339282
Norm type: 2


In [24]:
best_hyp_4 = {'norm': 2, 'alpha': 0.9970007612278947, 'beta': 0.5483462470349123, 'gamma': 0.4976855704268437}
recommender_slimer_user_ease = ThreeDifferentModelRecommender(URM_train.tocsr(), recommender_slimer, recommender_user, recommender_ease)
recommender_slimer_user_ease.fit(**best_hyp_4)
recommender_slimer_user_ease.RECOMMENDER_NAME = 'SLIMER+UserKNN+ease'

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.48800042232676766
UserKNNCFRecommender with weight beta: 0.26839819039332585
EASE with weight gamma: 0.24360138727990654
Norm type: 2


In [33]:
all_rec = [recommender_slim_all, recommender_slim_all2, recommender_ease, recommender_bpr, recommender_ials_hyb, recommender_slimer, recommender_slim_all2_ease, recommender_slim_all_ease, recommender_slimer_ease, recommender_ials_slimer_ease, recommender_slim_all2_ease_stk, recommender_slim_all_ease_stk, recommender_slimer_ease_stk, recommender_rp3_slimer_ease_stk, recommender_rp3_slimer_ease, recommender_slimer_bpr_ease, recommender_slimer_user_ease]
dict_res = rank_models_all(evaluator_validation, all_rec)
dict_res2 = rank_models_group_wise(evaluators, all_rec)

EvaluatorHoldout: Processed 13650 (100.0%) in 42.11 sec. Users per second: 324
             MAP
cutoff          
10      0.343051
EvaluatorHoldout: Processed 13650 (100.0%) in 28.99 sec. Users per second: 471
             MAP
cutoff          
10      0.342587
EvaluatorHoldout: Processed 13650 (100.0%) in 46.26 sec. Users per second: 295
             MAP
cutoff          
10      0.339313
EvaluatorHoldout: Processed 13650 (100.0%) in 31.08 sec. Users per second: 439
             MAP
cutoff          
10      0.331161
EvaluatorHoldout: Processed 13650 (100.0%) in 5.22 sec. Users per second: 2617
             MAP
cutoff          
10      0.322071
EvaluatorHoldout: Processed 13650 (100.0%) in 15.51 sec. Users per second: 880
             MAP
cutoff          
10      0.343213
EvaluatorHoldout: Processed 13650 (100.0%) in 49.48 sec. Users per second: 276
             MAP
cutoff          
10      0.344343
EvaluatorHoldout: Processed 13650 (100.0%) in 1.19 min. Users per second: 191
            

             MAP
cutoff          
10      0.414699
EvaluatorHoldout: Processed 1648 (100.0%) in 14.48 sec. Users per second: 114
             MAP
cutoff          
10      0.414107
EvaluatorHoldout: Processed 1648 (100.0%) in 15.26 sec. Users per second: 108
             MAP
cutoff          
10      0.413455
EvaluatorHoldout: Processed 1648 (100.0%) in 22.92 sec. Users per second: 72
             MAP
cutoff          
10      0.414309
EvaluatorHoldout: Processed 1648 (100.0%) in 23.58 sec. Users per second: 70
             MAP
cutoff          
10      0.415222
EvaluatorHoldout: Processed 1648 (100.0%) in 1.17 min. Users per second: 24
             MAP
cutoff          
10      0.413274
EvaluatorHoldout: Processed 1648 (100.0%) in 36.22 sec. Users per second: 46
             MAP
cutoff          
10      0.415749


In [34]:
for i in dict_res2.keys():
    dict_res2[i] = {k: v for k, v in sorted(dict_res2[i].items(), key=lambda item: item[1])}
    print(i, dict_res2[i])
    
dict_res = {k: v for k, v in sorted(dict_res.items(), key=lambda item: item[1])}
print(dict_res)

0 {'IALS_Hyb': 0.3662825656417167, 'SLIMall': 0.3728790769033312, 'EASE': 0.3831399885079597, 'SLIMall2': 0.3835481455230635, 'SLIMER': 0.38444432777398574, 'SLIMER+EASE': 0.3847280994965662, 'SLIMER+ease+RP3': 0.38483627488726646, 'SLIM_ALL+EASE': 0.3850657583871477, 'SLIM_ALL+EASE_HYB': 0.38511180675871864, 'SLIMER+EASE_HYB': 0.38548925756753705, 'SLIM_ALL2+EASE': 0.3854967463525895, 'SLIMER+ease+IALS': 0.3855705404175632, 'SLIMER+ease_stk+RP3': 0.38598508514026564, 'SLIM_ALL2+EASE_HYB': 0.3860021627786236, 'SLIMER+UserKNN+ease': 0.3861994160643547, 'SLIMER+BPR+ease': 0.38862192500422876}
1 {'IALS_Hyb': 0.2690387328203639, 'SLIMall': 0.2807495584107635, 'EASE': 0.2853267215931097, 'SLIMall2': 0.29134750653310015, 'SLIMER+ease+RP3': 0.2915975791231132, 'SLIMER': 0.2916087821815716, 'SLIMER+EASE': 0.2916331603271396, 'SLIMER+ease+IALS': 0.2917636348238483, 'SLIM_ALL+EASE': 0.29177717286101457, 'SLIMER+ease_stk+RP3': 0.29213007525164525, 'SLIMER+UserKNN+ease': 0.2923263525938827, 'SLIM_

In [35]:
dict_resA = rank_models_all(evaluator_validation, [recommender_ease_hyb])
dict_res2A = rank_models_group_wise(evaluators, [recommender_ease_hyb])

print(dict_resA)
print(dict_res2A)

EvaluatorHoldout: Processed 13650 (100.0%) in 29.70 sec. Users per second: 460
             MAP
cutoff          
10      0.340719
EvaluatorHoldout: Processed 5442 (100.0%) in 16.10 sec. Users per second: 338
            MAP
cutoff         
10      0.38434
EvaluatorHoldout: Processed 6560 (100.0%) in 6.60 sec. Users per second: 994
             MAP
cutoff          
10      0.287344
EvaluatorHoldout: Processed 1648 (100.0%) in 8.29 sec. Users per second: 199
             MAP
cutoff          
10      0.409134
{'EASE_Hybrid': 0.34071862026861927}
{0: {'EASE_Hybrid': 0.38433975458369096}, 1: {'EASE_Hybrid': 0.28734434886759674}, 2: {'EASE_Hybrid': 0.40913428012790837}}


In [41]:
rec_names = ['SLIMER+UserKNN+ease', 'SLIMER+BPR+ease', 'SLIMER+ease_stk+RP3', 'SLIMER+ease+RP3', 'SLIMER+ease+IALS']
d_redo = dict(dict_res)
d_redo2 = dict(dict_res2)

for rec in rec_names:
    del d_redo[rec]
    for i in dict_res2.keys():
        dict_rem = d_redo2[i]
    
for i in d_redo2.keys():
    print(i, d_redo2[i])
    
print(d_redo)

0 {'IALS_Hyb': 0.3662825656417167, 'SLIMall': 0.3728790769033312, 'EASE': 0.3831399885079597, 'SLIMall2': 0.3835481455230635, 'SLIMER': 0.38444432777398574, 'SLIMER+EASE': 0.3847280994965662, 'SLIM_ALL+EASE': 0.3850657583871477, 'SLIM_ALL+EASE_HYB': 0.38511180675871864, 'SLIMER+EASE_HYB': 0.38548925756753705, 'SLIM_ALL2+EASE': 0.3854967463525895, 'SLIM_ALL2+EASE_HYB': 0.3860021627786236}
1 {'IALS_Hyb': 0.2690387328203639, 'SLIMall': 0.2807495584107635, 'EASE': 0.2853267215931097, 'SLIMall2': 0.29134750653310015, 'SLIMER': 0.2916087821815716, 'SLIMER+EASE': 0.2916331603271396, 'SLIM_ALL+EASE': 0.29177717286101457, 'SLIM_ALL2+EASE': 0.29234996249516076, 'SLIMER+EASE_HYB': 0.29249604989353506, 'SLIM_ALL+EASE_HYB': 0.2926390280197436, 'SLIM_ALL2+EASE_HYB': 0.2929479166666668}
2 {'IALS_Hyb': 0.38717524368161554, 'SLIMall': 0.3940655099013716, 'EASE': 0.4094841029049163, 'SLIMall2': 0.4112873709354289, 'SLIMER': 0.4124746445908452, 'SLIMER+EASE_HYB': 0.4134550055863778, 'SLIMER+EASE': 0.4134

In [39]:
def test_percentage_sco(recommender_a, recommender_b, evaluator, high1, low1, step, norm=None):
    recommender = TwoDifferentModelRecommender(URM_train, recommender_a, recommender_b)
    results = []
    alp_space = np.linspace(high1, low1, step, False)
    for alp in alp_space:
        recommender.fit(norm, alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [26]:
recommender_hyb = TwoDifferentModelRecommender(URM_train, recommender_slim_all2, recommender_ease_hyb)
recommender_hyb.fit(None, 0.525)
rank_models(evaluator_validation, [recommender_hyb])

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
EvaluatorHoldout: Processed 13650 (100.0%) in 33.18 sec. Users per second: 411
DifferentLossScoresHybridRecommender 0.34476470434327483


In [33]:
alp_space, results = test_percentage_sco(recommender_slim_all2, recommender_ease_hyb, evaluator_validation, 0.525, 0.525, 1)

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
EvaluatorHoldout: Processed 13650 (100.0%) in 32.57 sec. Users per second: 419
0.525 : PRECISION                     0.501355
PRECISION_RECALL_MIN_DEN      0.502083
RECALL                        0.061162
MAP                           0.344765
MAP_MIN_DEN                   0.345092
MRR                            0.72598
NDCG                          0.514315
F1                            0.109024
HIT_RATE                      0.983956
ARHR_ALL_HITS                  1.53875
NOVELTY                       0.005479
AVERAGE_POPULARITY            0.573839
DIVERSITY_MEAN_INTER_LIST      0.90099
DIVERSITY_HERFINDAHL          0.990092
COVERAGE_ITEM                  0.07409
COVERAGE_ITEM_CORRECT         0.052661
COVERAGE_USER                      1.0
COVERAGE_USER_CORRECT         0.983956
DIVERSITY_GINI                 0.00948
SHANNON_ENTROPY                7.65926
RATIO_DIVERSITY_HERFINDAHL    0.990475
RATIO

In [40]:
alp_space, results = test_percentage_sco(recommender_slim_all2, recommender_ease_hyb, evaluator_validation, 0.6666666666666666, 0.6666666666666666, 30)

DifferentLossScoresHybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
EvaluatorHoldout: Processed 13650 (100.0%) in 31.80 sec. Users per second: 429
0.6666666666666666 : PRECISION                     0.501209
PRECISION_RECALL_MIN_DEN       0.50195
RECALL                        0.061158
MAP                           0.344784
MAP_MIN_DEN                   0.345117
MRR                           0.726602
NDCG                           0.51435
F1                            0.109014
HIT_RATE                      0.983956
ARHR_ALL_HITS                 1.539351
NOVELTY                       0.005498
AVERAGE_POPULARITY            0.565254
DIVERSITY_MEAN_INTER_LIST     0.905504
DIVERSITY_HERFINDAHL          0.990544
COVERAGE_ITEM                 0.077911
COVERAGE_ITEM_CORRECT          0.05482
COVERAGE_USER                      1.0
COVERAGE_USER_CORRECT         0.983956
DIVERSITY_GINI                 0.01011
SHANNON_ENTROPY               7.746769
RATIO_DIVERSITY_HERFINDAHL    0

KeyboardInterrupt: 

In [23]:
from Recommenders.HybridScores.DifferentStructure import ThreeDifferentModelRecommender
import optuna

class Objective(object):
    def __init__(self, URM_train, recommender1, recommender2, recommender3, evaluator):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train = URM_train
        self.evaluator = evaluator
        self.recommender_1 = recommender1
        self.recommender_2 = recommender2
        self.recommender_3 = recommender3

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        w_1 = trial.suggest_uniform('w_1', 0, 1)
        w_2 = trial.suggest_uniform('w_2', 0, 1)
        w_3 = trial.suggest_uniform('w_3', 0, 1)

        recommender_final = ThreeDifferentModelRecommender(self.URM_train.tocsr(), self.recommender_1, self.recommender_2, self.recommender_3)
        recommender_final.fit(2, w_1, w_2, w_3)
        result_dict, _ = self.evaluator.evaluateRecommender(recommender_final)

        map_v = -result_dict.loc[10]['MAP']
        if map_v >= -0.245: # minimum acceptable map
            # Calculate the penalty.
            trial.report(map_v, 0)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return map_v

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(Objective(URM_train, recommender_slim_all2, recommender_ease, recommender_ials_hyb, evaluator_validation), n_trials=500)

print(study.best_params)

[I 2022-01-07 02:03:10,104] A new study created in memory with name: no-name-6e2c214b-06a5-4312-aadb-38f1055699ff


CURRENT CONFIGURATION:
SLIMall2 with weight alpha: 0.6009300546474611
EASE with weight beta: 0.0028776896659438683
IALS_Hyb with weight gamma: 0.39619225568659494
Norm type: 2
EvaluatorHoldout: Processed 13649 (100.0%) in 4.91 min. Users per second: 46


[I 2022-01-07 02:08:04,867] Trial 0 finished with value: -0.3406633429741343 and parameters: {'w_1': 0.7656705058516254, 'w_2': 0.0036665866271239356, 'w_3': 0.504805380393256}. Best is trial 0 with value: -0.3406633429741343.


             MAP
cutoff          
10      0.340663
CURRENT CONFIGURATION:
SLIMall2 with weight alpha: 0.46166912528854026
EASE with weight beta: 0.32781583622269994
IALS_Hyb with weight gamma: 0.21051503848875977
Norm type: 2
EvaluatorHoldout: Processed 13649 (100.0%) in 4.75 min. Users per second: 48


[I 2022-01-07 02:12:49,782] Trial 1 finished with value: -0.34326919409178236 and parameters: {'w_1': 0.9170046494692686, 'w_2': 0.6511343937024024, 'w_3': 0.41814203831963836}. Best is trial 1 with value: -0.34326919409178236.


...

             MAP
cutoff          
10      0.343985
CURRENT CONFIGURATION:
SLIMall2 with weight alpha: 0.33803251215847685
EASE with weight beta: 0.31928084799383494
IALS_Hyb with weight gamma: 0.3426866398476882
Norm type: 2
EvaluatorHoldout: Processed 13649 (100.0%) in 4.87 min. Users per second: 47


[I 2022-01-07 09:19:03,347] Trial 90 finished with value: -0.3413301718714223 and parameters: {'w_1': 0.8742852209305081, 'w_2': 0.8257860314817943, 'w_3': 0.8863226283059659}. Best is trial 19 with value: -0.3443431055475898.


            MAP
cutoff         
10      0.34133
CURRENT CONFIGURATION:
SLIMall2 with weight alpha: 0.5101624368767683
EASE with weight beta: 0.43291787293783274
IALS_Hyb with weight gamma: 0.056919690185398976
Norm type: 2
EvaluatorHoldout: Processed 13000 (95.2%) in 5.07 min. Users per second: 43
EvaluatorHoldout: Processed 13649 (100.0%) in 5.12 min. Users per second: 44


[I 2022-01-07 09:24:10,824] Trial 91 finished with value: -0.3442597806455986 and parameters: {'w_1': 0.7735138585410675, 'w_2': 0.6563948070689163, 'w_3': 0.08630225590855341}. Best is trial 19 with value: -0.3443431055475898.


            MAP
cutoff         
10      0.34426
CURRENT CONFIGURATION:
SLIMall2 with weight alpha: 0.46786377234977916
EASE with weight beta: 0.4755639313311772
IALS_Hyb with weight gamma: 0.05657229631904361
Norm type: 2


In [22]:
best_hyp = {'norm': 2, 'alpha': 0.4583808522548663, 'beta': 0.09643193702948973, 'gamma': 0.10698567245567658}
recommender_rp3_slim_all2_ease = ThreeDifferentModelRecommender(URM_train, recommender_slim_all2, recommender_ease, recommender_rp3)
recommender_rp3_slim_all2_ease.fit(**best_hyp)
recommender_rp3_slim_all2_ease.RECOMMENDER_NAME = 'SLIMALL2+ease+RP3'

CURRENT CONFIGURATION:
SLIMall2 with weight alpha: 0.692629068749524
EASE with weight beta: 0.14571193891255987
RP3 with weight gamma: 0.16165899233791609
Norm type: 2


In [13]:
import numpy as np

def test_percentage_sim(recommender_a, recommender_b, evaluator, high1, low1, step):
    recommender = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_a.W_sparse, recommender_b.W_sparse)
    results = []
    alp_space = np.linspace(high1, low1, step, False)
    for alp in alp_space:
        recommender.fit(alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

def test_percentage_sco(recommender_a, recommender_b, evaluator, high1, low1, step, norm=None):
    recommender = TwoDifferentModelRecommender(URM_train, recommender_a, recommender_b)
    results = []
    alp_space = np.linspace(high1, low1, step, False)
    for alp in alp_space:
        recommender.fit(norm, alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [16]:
alp_space, results = test_percentage_sco(recommender_slim_all2, recommender_ease, evaluator_validation, 0.5, 0.66, 10)

EvaluatorHoldout: Processed 13649 (100.0%) in 1.10 min. Users per second: 207
            MAP
cutoff         
10      0.34418
0.5 : 0.34417953754388747
EvaluatorHoldout: Processed 13649 (100.0%) in 53.12 sec. Users per second: 257
             MAP
cutoff          
10      0.344018
0.516 : 0.3440182140211462
EvaluatorHoldout: Processed 13649 (100.0%) in 1.09 min. Users per second: 209
             MAP
cutoff          
10      0.344118
0.532 : 0.34411809923862224
EvaluatorHoldout: Processed 13649 (100.0%) in 1.11 min. Users per second: 204
             MAP
cutoff          
10      0.344143
0.548 : 0.34414300367373996
EvaluatorHoldout: Processed 13649 (100.0%) in 1.09 min. Users per second: 209
             MAP
cutoff          
10      0.344114
0.5640000000000001 : 0.3441144970211216
EvaluatorHoldout: Processed 13649 (100.0%) in 1.01 min. Users per second: 225
             MAP
cutoff          
10      0.344264
0.5800000000000001 : 0.34426407481448235
EvaluatorHoldout: Processed 13649 (100

In [17]:
alp_space, results = test_percentage_sco(recommender_slimer, recommender_ease, evaluator_validation, 0.1, 0.9, 100)

EvaluatorHoldout: Processed 13649 (100.0%) in 1.10 min. Users per second: 207
             MAP
cutoff          
10      0.340423
0.1 : 0.3404232183996273
EvaluatorHoldout: Processed 13649 (100.0%) in 1.13 min. Users per second: 201
             MAP
cutoff          
10      0.340586
0.10800000000000001 : 0.340586231097808
EvaluatorHoldout: Processed 13649 (100.0%) in 1.11 min. Users per second: 204
             MAP
cutoff          
10      0.340734
0.116 : 0.3407344540619891
EvaluatorHoldout: Processed 13649 (100.0%) in 1.09 min. Users per second: 208
             MAP
cutoff          
10      0.340935
0.124 : 0.3409347768951054
EvaluatorHoldout: Processed 13649 (100.0%) in 1.13 min. Users per second: 202
          MAP
cutoff       
10      0.341
0.132 : 0.3410003000394253
EvaluatorHoldout: Processed 13649 (100.0%) in 1.11 min. Users per second: 205
             MAP
cutoff          
10      0.341081
0.14 : 0.34108063326925625
EvaluatorHoldout: Processed 13649 (100.0%) in 1.14 min. Users 

EvaluatorHoldout: Processed 13649 (100.0%) in 1.13 min. Users per second: 201
             MAP
cutoff          
10      0.343831
0.508 : 0.34383097139507973
EvaluatorHoldout: Processed 13649 (100.0%) in 1.08 min. Users per second: 210
             MAP
cutoff          
10      0.343876
0.516 : 0.34387632909905663
EvaluatorHoldout: Processed 13649 (100.0%) in 1.08 min. Users per second: 212
             MAP
cutoff          
10      0.343847
0.524 : 0.3438467409089805
EvaluatorHoldout: Processed 13649 (100.0%) in 1.14 min. Users per second: 200
             MAP
cutoff          
10      0.343925
0.532 : 0.3439254983503654
EvaluatorHoldout: Processed 13649 (100.0%) in 1.13 min. Users per second: 201
            MAP
cutoff         
10      0.34393
0.54 : 0.34393037689836
EvaluatorHoldout: Processed 13649 (100.0%) in 1.11 min. Users per second: 205
            MAP
cutoff         
10      0.34383
0.548 : 0.3438298695061105
EvaluatorHoldout: Processed 13649 (100.0%) in 1.15 min. Users per secon

In [19]:
alp_space, results = test_percentage_sco(recommender_slim_all2, recommender_ease, evaluator_validation, 0.4, 0.7, 30)

EvaluatorHoldout: Processed 13649 (100.0%) in 1.12 min. Users per second: 203
             MAP
cutoff          
10      0.343934
0.4 : 0.3439335197531783
EvaluatorHoldout: Processed 13649 (100.0%) in 1.14 min. Users per second: 199
             MAP
cutoff          
10      0.343854
0.41000000000000003 : 0.3438539802322881
EvaluatorHoldout: Processed 13649 (100.0%) in 1.07 min. Users per second: 213
             MAP
cutoff          
10      0.343872
0.42000000000000004 : 0.3438717412869382
EvaluatorHoldout: Processed 13649 (100.0%) in 1.11 min. Users per second: 204
             MAP
cutoff          
10      0.343876
0.43 : 0.3438761546575322
EvaluatorHoldout: Processed 13649 (100.0%) in 1.09 min. Users per second: 208
             MAP
cutoff          
10      0.343957
0.44 : 0.3439573600949911
EvaluatorHoldout: Processed 13649 (100.0%) in 1.11 min. Users per second: 205
             MAP
cutoff          
10      0.343921
0.45 : 0.34392144839961747
EvaluatorHoldout: Processed 13649 (100.0

In [13]:
ofp = "../all_models/URM_all/"

recommender1a = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_all, ICM_all)
recommender2a = EASE_R_Recommender(URM_all)
recommender3a = ItemKNNCBFRecommender(URM_all, ICM_all)

model_init(recommender1a, 'SLIMall', models_to_combine_best['SLIM_all'])
model_init(recommender2a, 'EASE', models_to_combine_best['EASE'])
model_init(recommender3a, 'ItemKNNall', models_to_combine_best['icm_all'])

../all_models/URM_all/SLIMall.zip
Model does not exists, creating...


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [19:19<00:00, 15.57it/s]


SLIMElasticNetRecommender: Saving model in file '../all_models/URM_all/SLIMall'
SLIMElasticNetRecommender: Saving complete
../all_models/URM_all/EASE.zip
Model does not exists, creating...
EASE_R_Recommender: Fitting model... 
Similarity column 18059 (100.0%), 514.04 column/sec. Elapsed time 35.13 sec
EASE_R_Recommender: Fitting model... done in 4.00 min
EASE_R_Recommender: Saving model in file '../all_models/URM_all/EASE'
EASE_R_Recommender: Saving complete


In [65]:
ofp = "../all_models/URM_all/"

recommender1a = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_all, ICM_all)
recommender2a = EASE_R_Recommender_Hybrid(URM_all, ICM_all)
recommender3a = UserKNNCFRecommender(URM_all)

model_init(recommender1a, 'SLIMall2', models_to_combine_best['SLIM_all2'], "")
model_init(recommender2a, 'EASE_Hybrid', models_to_combine_best['EASE_stk'], "")
model_init(recommender3a, 'UserKNN', models_to_combine_best['UserKNN'], "")

../all_models/URM_all/SLIMall2.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../all_models/URM_all/SLIMall2'
SLIMElasticNetRecommender: Loading complete
../all_models/URM_all/EASE_Hybrid.zip
Model found!
EASE_R_Recommender_Hybrid: Loading model from file '../all_models/URM_all/EASE_Hybrid'
EASE_R_Recommender_Hybrid: Loading complete
../all_models/URM_all/UserKNN.zip
Model found!
UserKNNCFRecommender: Loading model from file '../all_models/URM_all/UserKNN'
UserKNNCFRecommender: Loading complete


In [67]:
recommender_hyb = ItemKNNScoresHybridTwoRecommender_FAST(URM_all, recommender1a, recommender2a)
recommender_hyb.fit(0.525)

In [68]:
recommender_final = ItemKNNScoresHybridTwoRecommender(URM_all, recommender_hyb, recommender3a)
recommender_final.fit(0.91)

In [69]:
import pandas as pd
K=10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
                                sep=",",
                                dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
    subm_res["user_id"].append(user_id)
    res = recommender_final.recommend(user_id, K)
    print("USER", user_id, "RECOMMENDED!")
    res = ' '.join(map(str, res))
    print(res)
    subm_res["item_list"].append(res)


# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
# submission

from datetime import datetime
now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)

USER 0 RECOMMENDED!
3899 5433 8465 16365 16359 2243 640 6061 9130 8635
USER 1 RECOMMENDED!
3473 14400 940 16365 16012 640 2118 10118 14358 9275
USER 2 RECOMMENDED!
6177 1551 17022 17330 16365 14229 15119 4607 2748 5925
USER 3 RECOMMENDED!
8194 15551 15082 16359 960 8371 16444 15507 7143 5885
USER 4 RECOMMENDED!
8194 6177 2152 10954 10586 3473 5299 9871 6011 16494
USER 5 RECOMMENDED!
1846 16640 14396 9621 7867 11227 13098 13585 99 5312
USER 6 RECOMMENDED!
6214 985 16846 14358 14181 8207 7606 13443 9621 14868
USER 7 RECOMMENDED!
13963 15119 1885 3433 4292 14031 7751 17364 640 3768
USER 8 RECOMMENDED!
8635 3619 5605 2665 1259 10865 3768 5299 6011 8502
USER 9 RECOMMENDED!
5605 8207 1614 13318 14078 14181 16755 15117 1551 3473
USER 10 RECOMMENDED!
3899 14358 6214 13229 6975 14400 4607 16365 5273 13410
USER 11 RECOMMENDED!
14181 10865 3473 11359 6431 12390 249 5577 1836 13134
USER 12 RECOMMENDED!
3899 6724 3200 13963 1792 1137 15778 5925 1619 13134
USER 13 RECOMMENDED!
8635 6214 15119 6975 1

USER 137 RECOMMENDED!
14358 3899 6011 7143 15844 960 5981 2886 4582 14396
USER 138 RECOMMENDED!
8635 8096 3899 7257 4301 9686 6214 5925 13638 1838
USER 139 RECOMMENDED!
3899 8635 7407 1941 2756 13963 5925 1551 6214 249
USER 140 RECOMMENDED!
6214 6652 13914 8207 16365 15778 1137 13410 6975 15583
USER 141 RECOMMENDED!
640 6214 8207 6011 16494 6589 12901 13410 5299 4582
USER 142 RECOMMENDED!
15119 17584 640 4582 10562 3768 14917 12457 4292 4088
USER 143 RECOMMENDED!
1941 3899 14181 14485 7309 10370 15778 8326 16486 9871
USER 144 RECOMMENDED!
16070 8635 1653 13725 1259 15778 1614 6782 4078 6214
USER 145 RECOMMENDED!
13134 10865 13465 1137 15674 1941 12281 1044 11553 13691
USER 146 RECOMMENDED!
17788 8194 5885 5981 10871 15551 15082 3473 16359 640
USER 147 RECOMMENDED!
15119 6177 1792 17364 1619 8194 13410 14781 9131 985
USER 148 RECOMMENDED!
6177 17135 10865 3889 10295 13638 1792 640 17748 12752
USER 149 RECOMMENDED!
6214 16846 12492 12186 8635 1259 16791 11339 9131 14093
USER 150 RECOMMEN

USER 275 RECOMMENDED!
6157 6214 13295 11491 17330 14181 2251 2796 1551 6670
USER 276 RECOMMENDED!
1614 13410 10562 3942 8635 10865 3619 13134 11339 13178
USER 277 RECOMMENDED!
442 14520 7257 3473 5237 16762 2711 4435 8780 7825
USER 278 RECOMMENDED!
16486 7264 13914 15119 249 8567 6782 10370 7486 10562
USER 279 RECOMMENDED!
16846 6214 5324 15119 13229 640 14781 765 14868 3889
USER 280 RECOMMENDED!
6177 8651 1614 8942 3130 103 16791 16012 17364 8573
USER 281 RECOMMENDED!
10010 15026 5141 8194 10131 8011 7257 14311 16846 1887
USER 282 RECOMMENDED!
14358 4292 17788 9131 3768 6975 6011 6118 17294 3869
USER 283 RECOMMENDED!
1551 15778 7716 15551 6670 17330 13134 13229 10865 1208
USER 284 RECOMMENDED!
4293 6214 15735 9939 2126 4582 8318 10865 15778 1259
USER 285 RECOMMENDED!
2665 8687 8898 13950 6096 16365 899 6676 202 4078
USER 286 RECOMMENDED!
17330 15778 3130 16946 249 6652 3200 955 8651 16791
USER 287 RECOMMENDED!
16846 10491 1619 8622 640 14255 9250 14814 4582 15181
USER 288 RECOMMENDED!

USER 418 RECOMMENDED!
16365 17330 640 2665 5273 13410 10865 13134 2886 6975
USER 419 RECOMMENDED!
3473 8194 17297 8635 9131 6118 14996 8911 7486 15778
USER 420 RECOMMENDED!
2665 6214 15181 16846 16365 3899 8635 955 17294 11342
USER 421 RECOMMENDED!
15119 16846 3768 1619 985 5324 6011 14358 15181 10316
USER 422 RECOMMENDED!
9871 14400 12311 5424 3402 16012 15750 15683 249 17961
USER 423 RECOMMENDED!
17330 11576 13134 8326 6905 5675 14536 8573 12956 202
USER 424 RECOMMENDED!
6177 5273 8635 5299 3768 4582 10562 5324 16486 1259
USER 425 RECOMMENDED!
6214 17330 16365 5324 15119 16791 4204 17364 7716 17748
USER 426 RECOMMENDED!
10865 15119 17364 13725 8635 14400 14181 5742 6011 13178
USER 427 RECOMMENDED!
12901 6975 16624 10865 6589 14226 3200 3942 1137 11110
USER 428 RECOMMENDED!
13914 15119 5925 15778 1941 10295 16846 13387 10562 1208
USER 429 RECOMMENDED!
7870 17294 8502 6782 17936 4582 13032 11836 16070 13914
USER 430 RECOMMENDED!
3473 10562 14868 6306 17022 17748 10370 1208 14627 9527
U

USER 550 RECOMMENDED!
78 4853 9744 840 7486 6431 17135 4582 15683 2772
USER 551 RECOMMENDED!
6177 10370 8635 10562 11339 10865 17411 14093 13178 15469
USER 552 RECOMMENDED!
7716 14400 8635 6177 15796 3899 8895 15289 13914 1137
USER 553 RECOMMENDED!
10132 9871 8635 13575 12958 1651 3473 1614 5273 981
USER 554 RECOMMENDED!
3899 3768 1792 16624 447 9482 4582 15386 13410 16436
USER 555 RECOMMENDED!
3899 16846 10865 3768 640 14358 2772 11430 11303 13229
USER 556 RECOMMENDED!
5273 16365 2772 7870 1259 8371 13229 8207 16012 4582
USER 557 RECOMMENDED!
11702 16919 13178 14093 17748 7973 7407 14181 10370 6177
USER 558 RECOMMENDED!
14762 14237 3204 15026 7257 4791 202 6214 13703 9323
USER 559 RECOMMENDED!
12707 8651 1941 13134 13914 16791 14358 16364 14093 6975
USER 560 RECOMMENDED!
4582 13032 9131 3899 17936 5299 10865 16070 1533 13134
USER 561 RECOMMENDED!
13963 14917 6652 17574 5974 3899 3889 8635 17140 15119
USER 562 RECOMMENDED!
13178 1259 16486 2665 13563 3889 6177 15082 1208 10295
USER 563

...

USER 13044 RECOMMENDED!
8635 17936 6177 512 249 16012 1137 13691 1386 13178
USER 13045 RECOMMENDED!
6177 15778 12828 5478 17553 14426 3942 5605 14408 4173
USER 13046 RECOMMENDED!
8502 11836 7224 1137 15756 10118 7763 3847 3473 6782
USER 13047 RECOMMENDED!
447 6214 3473 8635 1551 3768 9482 12492 1792 16291
USER 13048 RECOMMENDED!
15750 3473 8502 7407 17135 249 11359 17416 12707 3942
USER 13049 RECOMMENDED!
6905 17135 10095 11359 1208 5273 12530 3473 14917 13914
USER 13050 RECOMMENDED!
5424 5403 2772 249 4204 10914 1933 6214 7224 7760
USER 13051 RECOMMENDED!
3473 10865 17294 3899 2665 14358 12901 4582 6214 16365
USER 13052 RECOMMENDED!
6177 14181 17330 8207 17294 13914 12457 13229 13443 10865
USER 13053 RECOMMENDED!
3895 7735 8313 2067 15376 5561 13096 8713 1551 16598
USER 13054 RECOMMENDED!
3473 8635 7264 16365 16846 17330 7744 15119 13134 5925
USER 13055 RECOMMENDED!
15119 640 8635 3942 5640 14868 17330 1137 1792 14485
USER 13056 RECOMMENDED!
12390 840 3889 1515 1137 13178 12492 5925 2

USER 13262 RECOMMENDED!
17830 5925 16365 13410 14358 3579 10954 1259 17330 15187
USER 13263 RECOMMENDED!
985 15119 14358 1267 4607 13229 3433 4292 7744 8635
USER 13264 RECOMMENDED!
552 12186 6975 13229 17364 9131 5324 2665 3473 16624
USER 13265 RECOMMENDED!
6157 17715 8635 552 14358 17958 15289 8854 11518 14974
USER 13266 RECOMMENDED!
6177 3829 5974 17135 13914 10865 6905 4204 1614 5605
USER 13267 RECOMMENDED!
6652 4292 13410 3433 6177 16365 17140 53 1267 3955
USER 13268 RECOMMENDED!
12186 17364 5299 14509 16751 2152 10316 14181 3691 7983
USER 13269 RECOMMENDED!
12901 16365 6214 3473 8011 4582 1792 8207 1619 3200
USER 13270 RECOMMENDED!
7309 1941 8567 1551 5273 3473 3200 8635 5605 13856
USER 13271 RECOMMENDED!
6976 2807 17905 8580 1614 13295 17364 13134 13296 3869
USER 13272 RECOMMENDED!
1208 8635 6905 14400 1137 10746 6782 10295 7407 14996
USER 13273 RECOMMENDED!
14358 1941 16436 2152 8635 4582 17330 13410 5141 1792
USER 13274 RECOMMENDED!
13229 15750 4088 9744 16791 14358 12707 2152 

USER 13368 RECOMMENDED!
1941 15119 15778 8635 13032 5925 17330 12901 4582 6905
USER 13369 RECOMMENDED!
3473 8635 17330 15778 9871 2665 3691 2076 12186 8473
USER 13370 RECOMMENDED!
10954 3768 2152 2665 6652 6011 10562 3130 13914 3889
USER 13371 RECOMMENDED!
16365 16846 15119 15181 5324 6589 2886 3768 1792 3899
USER 13372 RECOMMENDED!
13914 1259 14868 8651 12457 8635 1551 10562 3829 1941
USER 13373 RECOMMENDED!
7295 3899 3569 14868 15750 512 3829 17022 7224 9441
USER 13374 RECOMMENDED!
985 6214 17294 16846 3473 14358 8207 2665 12186 3433
USER 13375 RECOMMENDED!
12457 12028 11051 1941 13318 15082 6670 17788 10370 10758
USER 13376 RECOMMENDED!
7867 15289 3768 10865 1259 1551 5299 4582 12186 3829
USER 13377 RECOMMENDED!
16365 640 12901 2846 15778 1883 7744 1619 4582 6011
USER 13378 RECOMMENDED!
16365 3768 15119 3473 12901 8635 5324 15117 13856 552
USER 13379 RECOMMENDED!
4292 6011 3473 2665 8635 6975 14358 10954 13654 1605
USER 13380 RECOMMENDED!
15119 7744 16946 2665 4292 1137 12901 13229 

USER 13473 RECOMMENDED!
3899 6214 16365 5742 6476 15119 17330 1208 5403 13229
USER 13474 RECOMMENDED!
17330 6177 16365 249 1551 17135 8207 241 6873 5273
USER 13475 RECOMMENDED!
6177 14781 17364 8207 2545 4292 3768 13032 17022 6652
USER 13476 RECOMMENDED!
2376 7295 17294 5299 5258 17296 16278 5424 5403 10253
USER 13477 RECOMMENDED!
4582 7412 8936 3869 4607 3473 5605 10865 8635 3829
USER 13478 RECOMMENDED!
14781 640 3473 2886 8635 16494 5403 11359 17140 14400
USER 13479 RECOMMENDED!
16486 10295 6905 11339 1208 6975 14868 1137 4204 13134
USER 13480 RECOMMENDED!
1551 3473 14181 17135 2772 249 16365 13229 2665 1533
USER 13481 RECOMMENDED!
11343 8473 3691 2796 1846 14396 3668 15426 15473 1114
USER 13482 RECOMMENDED!
16846 6214 3473 13229 2665 9131 5299 12186 10954 2182
USER 13483 RECOMMENDED!
6177 14181 5742 14400 3473 6905 10370 8635 6214 17356
USER 13484 RECOMMENDED!
16919 2886 3768 16365 4088 14536 13229 5324 16846 13032
USER 13485 RECOMMENDED!
9875 8635 3768 10865 8584 16486 10908 11391 

USER 13580 RECOMMENDED!
13296 2807 4119 11058 16846 14918 2482 4520 14010 14358
USER 13581 RECOMMENDED!
13318 16486 8635 15181 16012 1792 13638 13914 16365 17135
USER 13582 RECOMMENDED!
3899 14358 13032 16846 2886 640 11175 6011 13914 1885
USER 13583 RECOMMENDED!
16365 5273 6011 15778 1792 9131 14485 3829 640 16486
USER 13584 RECOMMENDED!
11359 8207 15119 7295 16846 14358 14078 10370 7264 12752
USER 13585 RECOMMENDED!
9131 2152 16791 6214 16359 16486 16808 3473 1259 7870
USER 13586 RECOMMENDED!
3473 1551 8635 17135 5403 8651 13914 14917 17748 17936
USER 13587 RECOMMENDED!
5925 5273 16364 17330 13914 13691 17526 17936 803 1551
USER 13588 RECOMMENDED!
8194 8207 15583 16846 5460 14181 7719 960 6975 16359
USER 13589 RECOMMENDED!
6177 16549 8011 7143 13229 3768 13134 13410 7975 3200
USER 13590 RECOMMENDED!
17584 3899 10562 7257 1551 8326 5925 8635 14181 11576
USER 13591 RECOMMENDED!
8635 17330 1137 11359 3829 6011 1792 4292 14181 10370
USER 13592 RECOMMENDED!
14396 16444 13295 14159 9898 81